In [1]:
import gdown

gdown.download('https://bit.ly/3q9SZix', '20s_best_book.json', quiet=False)

Downloading...
From: https://bit.ly/3q9SZix
To: C:\구영선\python\Python_Class(2023)\20s_best_book.json
100%|██████████████████████████████████████████████████████████████████████████████| 92.9k/92.9k [00:00<00:00, 790kB/s]


'20s_best_book.json'

In [2]:
import pandas as pd

books_df = pd.read_json('20s_best_book.json')
books_df.head(3)

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,461,https://image.aladin.co.kr/product/19359/16/co...
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,387,https://image.aladin.co.kr/product/24512/70/co...
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,03810,,813.7,문학 > 한국문학 > 소설,383,https://image.aladin.co.kr/product/19804/82/co...


In [3]:
books1 = books_df[['no','bookname','authors','publisher','publication_year','isbn13','loan_count','class_nm']]
books1.head()

,no,bookname,authors,publisher,publication_year,isbn13,loan_count,class_nm
0,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,461,문학 > 한국문학 > 소설
1,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,387,문학 > 한국문학 > 소설
2,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,383,문학 > 한국문학 > 소설
3,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,370,문학 > 한국문학 > 소설
4,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,365,문학 > 한국문학 > 소설


In [4]:
books2 = books1[['bookname','isbn13','loan_count']]
books2.head()

,bookname,isbn13,loan_count
0,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,9791190090018,461
1,달러구트 꿈 백화점.이미예 장편소설,9791165341909,387
2,지구에서 한아뿐 :정세랑 장편소설,9791188862290,383
3,"시선으로부터, :정세랑 장편소설",9788954672214,370
4,아몬드 :손원평 장편소설,9788936434267,365


In [5]:
type(books2)

pandas.core.frame.DataFrame

In [6]:
len(books2['bookname'])

200

In [7]:
import requests

requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

In [10]:
import requests

isbn = 9791190090018      # '우리가 빛의 속도로 갈 수 없다면'의 ISBN
url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'

r = requests.get(url.format(isbn))

In [11]:
from bs4 import BeautifulSoup

In [12]:
soup = BeautifulSoup(r.text, 'html.parser')

In [13]:
def get_wgt(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[2]
    return ''

In [14]:
get_wgt(9791190090018)

'496g'

In [15]:
top10_books = books1.head(10)

In [16]:
def get_wgt2(row):
    isbn = row['isbn13']
    return get_wgt(isbn)

In [17]:
book_wgt = top10_books.apply(get_wgt2, axis=1)
print(book_wgt)

0    496g
1    358g
2    296g
3    412g
4    388g
5    512g
6    318g
7        
8    324g
9    406g
dtype: object


In [18]:
book_wgt.name ='Book Weight'
print(book_wgt)

0    496g
1    358g
2    296g
3    412g
4    388g
5    512g
6    318g
7        
8    324g
9    406g
Name: Book Weight, dtype: object


In [19]:
top10_with_wgt = pd.merge(top10_books, book_wgt, 
                                  left_index=True, right_index=True)
top10_with_wgt

,no,bookname,authors,publisher,publication_year,isbn13,loan_count,class_nm,Book Weight
0,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,461,문학 > 한국문학 > 소설,496g
1,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,387,문학 > 한국문학 > 소설,358g
2,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,383,문학 > 한국문학 > 소설,296g
3,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,370,문학 > 한국문학 > 소설,412g
4,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,365,문학 > 한국문학 > 소설,388g
5,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,320,문학 > 한국문학 > 소설,512g
6,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,319,문학 > 한국문학 > 소설,318g
7,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,312,문학 > 일본문학 > 소설,
8,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,293,"사회과학 > 사회학, 사회문제 > 사회문제",324g
9,10,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,293,문학 > 한국문학 > 소설,406g
